In [2]:
import pandas as pd

In [3]:
df_historico = pd.read_csv('./datasets/1700665983594-attachment.csv')

In [4]:
df_historico.head(5)

,horas_estudo,pontuacao_teste
0,1.1,30
1,2.0,55
2,2.5,60
3,3.6,75
4,4.2,85
